In [1]:
import torch
import torch.autograd.gradcheck

import pyqtorch as pyq
from pyqtorch.embed import ConcretizedCallable, Embedding
# name0, fn0 = "fn0", ConcretizedCallable("sin", ["x"])
name1, fn1 = "fn1", ConcretizedCallable("mul", [3, "x"])
# name2, fn2 = "fn2", ConcretizedCallable("mul", ["fn1", 2.0])
# name3, fn3 = "fn3", ConcretizedCallable("log", ["fn2"])
embedding = pyq.Embedding(
    # vparam_names=["x"],
    fparam_names=["x"],
    var_to_call={name1: fn1},
    # var_to_call={name0: fn0, name1: fn1, name2: fn2, name3: fn3},
)
rx = pyq.RX(0, param_name=name1)
# cry = pyq.CRY(0, 1, param_name=name1)
# phase = pyq.PHASE(1, param_name=name2)
# ry = pyq.RY(1, param_name=name3)
# cnot = pyq.CNOT(1, 2)
ops = [rx]
# ops = [rx, cry, phase, ry, cnot]
n_qubits = 1
circ = pyq.QuantumCircuit(n_qubits, ops)
obs = pyq.Observable([pyq.Z(0)])

state = pyq.zero_state(n_qubits)

x = torch.tensor(0.1236, requires_grad=True) #torch.rand(1, requires_grad=True)
# y = torch.rand(1, requires_grad=True)

values_ad = {"x": x}
embedded_params = embedding(values_ad)
exp_val = pyq.expectation(circ, state, embedded_params, obs, embedding=embedding)

print(exp_val)

PyQTorch logger successfully setup with log level 20


tensor([0.9320], dtype=torch.float64, grad_fn=<SelectBackward0>)


In [6]:
torch.zeros_like(torch.tensor(1.0))

tensor(0.)

In [2]:
from importlib import import_module
a = import_module("pyqtorch.utils")

In [3]:
a.heaviside()

TypeError: heaviside() missing 2 required positional arguments: 'x' and 'vals'

In [8]:
embedded_params

{'x': tensor([0.3781], requires_grad=True),
 'y': tensor([0.0050], requires_grad=True),
 'fn0': tensor([0.3691], grad_fn=<SinBackward0>),
 'fn1': tensor([0.0019], grad_fn=<MulBackward0>),
 'fn2': tensor([0.0037], grad_fn=<MulBackward0>),
 'fn3': tensor([-5.5958], grad_fn=<LogBackward0>)}

In [2]:
from pyqtorch import zero_state
from pyqtorch.utils import SolverType

In [3]:
omega = 20.0
param_y = 2.0
param_x = 5.0
sigma_x = torch.tensor([[0.0, 1.0], [1.0, 0.0]])
sigma_y = torch.tensor([[0.0, -1.0j], [1.0j, 0.0]])


def hamiltonian_t(t: float) -> torch.Tensor:
    t = torch.as_tensor(t)
    return omega * (
        param_y * torch.sin(t) * torch.kron(sigma_x, torch.eye(2))
        + param_x * t**2 * torch.kron(torch.eye(2), sigma_y)
    ).to(torch.complex128)

In [4]:
hamiltonian_t(0.2)

tensor([[0.0000+0.0000j, 0.0000-4.0000j, 7.9468+0.0000j, 0.0000+0.0000j],
        [0.0000+4.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 7.9468+0.0000j],
        [7.9468+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 0.0000-4.0000j],
        [0.0000+0.0000j, 7.9468+0.0000j, 0.0000+4.0000j, 0.0000+0.0000j]],
       dtype=torch.complex128)

In [5]:
zero_state(2)

tensor([[[1.+0.j],
         [0.+0.j]],

        [[0.+0.j],
         [0.+0.j]]], dtype=torch.complex128)

In [6]:
import pyqtorch as pyq

In [13]:
sin_t, sin_fn = "sin_t", ConcretizedCallable("sin", ["t"])
t_sq, t_sq_fn = "t_sq", ConcretizedCallable("mul", [1.0, "t"])

generator = pyq.Scale(
    pyq.Add([
        pyq.Scale(
            pyq.Scale(
                pyq.X(0),
                sin_t
            ),
            param_y
        ),
        pyq.Scale(
            pyq.Scale(
                pyq.Y(1),
                t_sq
            ),
            param_x
        ),
    ]),
    omega)

In [14]:
from 

[]

In [9]:
torch.tensor(-1+0j) ** 0.5

tensor(0.+1.j)

In [9]:
expr_pyq = pyq.sqrt(pyq.sin("x")) + pyq.cos("r") * (1.0 / pyq.log("z") * "y")
# expr_pyq = pyq.cos("z") ** pyq.log(1.5 / (1.0 + (2.0 * pyq.sin("x")) + "y"))

In [21]:
def get_funcs(expr, independent_args=set()):
    fn_name = expr.call_name
    # print("function:", expr.call_name, "; function argument")
    abstract_args = expr.abstract_args
    print(f"[{fn_name}] args: {[arg.call_name if isinstance(arg, ConcretizedCallable) else arg for arg in abstract_args]}")
    if len(abstract_args) == 1 and isinstance(abstract_args[0], str):
        independent_args.add(abstract_args[0])
        print("[0] independent arg:", abstract_args[0])
        print("---------------")
    else:
        
        for arg in abstract_args:
            if isinstance(arg, ConcretizedCallable):
                # print("function:", arg.call_name)
                get_funcs(arg)
            else:
                print("[1] independent arg:", arg)
                print('---------------')

    return independent_args




In [37]:
def _get_independent_args(cc: ConcretizedCallable) -> set:
    # if out is None:
    out = set()
    if len(cc.abstract_args) == 1 and isinstance(cc.abstract_args[0], str):
        print("independent arg:", cc.abstract_args[0])
        return set([cc.abstract_args[0]])
    else:
        # out = set()
        for arg in cc.abstract_args:
            if isinstance(arg, ConcretizedCallable):
                res = _get_independent_args(arg)
                print("got:", res, "out:", out)
                out.add(res)
            else:
                print("arg:", arg, out)
                if isinstance(arg, str):
                    out.add(arg)
    return out

In [22]:
get_funcs(cc)

[mul] args: [3.0, 'x']
[1] independent arg: 3.0
---------------
[1] independent arg: x
---------------


set()

In [32]:
st = set()
st.add("a")
st.add("b")
st.add("a")

In [33]:
st

{'a', 'b'}

In [38]:
_get_independent_args(cc)

arg: 3.0 set()
arg: x set()


{'x'}

In [10]:
expr_pyq.independent_args

['z', 'x', 'r']

In [12]:
cc = ConcretizedCallable("mul", [3.0, "x"])
cc.independent_args

[]

In [12]:
get_funcs(expr_pyq)

NameError: name 'get_funcs' is not defined

In [12]:
from sympy import *
x, y, z = symbols('x y z')

In [13]:
expr_sympy = sqrt(sin(x))
# expr_sympy = cos(z) ** log(1.5 / (1.0 + (2 * sin(x)) + y))
srepr(expr_sympy)

"Pow(sin(Symbol('x')), Rational(1, 2))"

In [14]:
vals = {"x": torch.tensor(2.0), "y": torch.tensor(0.5), "z": torch.tensor(0.2)}
expr_pyq(vals).item()

0.9535708427429199

In [15]:
Rational("1/2").evalf()

0.500000000000000

In [16]:
expr_sympy.subs({x: vals["x"], y: vals["y"], z: vals["z"]})

0.953570881909511

In [17]:
from functools import reduce, partial

In [18]:
mapping = {
    Pow: "pow",
    cos: "cos",
    Add: "add",
    Mul: "mul",
    sin: "sin",
    log: "log",
    # sqrt: "sqrt",
    # Rational: 
}

In [19]:
def sympy_to_pyq(expr):
    print("recursion level start...")
    print("func:", expr.func, "|", expr.args)

    if len(expr.args) == 0:
        print("returning arg from base case:")
        res = float(expr) if str(expr).replace('.', '', 1).replace('-', '', 1).isdigit() else str(expr)
        if isinstance(res, str):
            if "/" in res:
                # found a rational
                res = float(Rational(res).evalf())
            if "fix" in res:
                # found a fixed parameter - convert to float
                res = float(res.split("_")[1])
        print(res, type(res))
        return res

    all_results = []
    for i, arg in enumerate(expr.args):
        print(f"--- {i} all func args:", expr.args)
        res = sympy_to_pyq(arg)

        print(f"[{i}] returned arg:", res)


        all_results.append(res)
    
    print("args:")
    print(all_results)

    # if isinstance(expr.func, Rational):


    if len(all_results) > 2:
        fn = lambda x, y: partial(ConcretizedCallable, call_name=mapping[expr.func])(abstract_args=[x, y])
        cc = reduce(fn, all_results)
    else:
        # a = ", ".join(tmp) if len(tmp) > 1 else str(tmp[0])
        cc = ConcretizedCallable(mapping[expr.func], all_results)

    print("Concretized callable object:")
    print(cc)
    return cc

In [20]:
generator_qadence = omega * (y * sin(x) + x * (z**2))

In [21]:
cc = sympy_to_pyq(generator_qadence)
cc(vals)

recursion level start...
func: <class 'sympy.core.add.Add'> | (20.0*x*z**2, 20.0*y*sin(x))
--- 0 all func args: (20.0*x*z**2, 20.0*y*sin(x))
recursion level start...
func: <class 'sympy.core.mul.Mul'> | (20.0000000000000, x, z**2)
--- 0 all func args: (20.0000000000000, x, z**2)
recursion level start...
func: <class 'sympy.core.numbers.Float'> | ()
returning arg from base case:
20.0 <class 'float'>
[0] returned arg: 20.0
--- 1 all func args: (20.0000000000000, x, z**2)
recursion level start...
func: <class 'sympy.core.symbol.Symbol'> | ()
returning arg from base case:
x <class 'str'>
[1] returned arg: x
--- 2 all func args: (20.0000000000000, x, z**2)
recursion level start...
func: <class 'sympy.core.power.Pow'> | (z, 2)
--- 0 all func args: (z, 2)
recursion level start...
func: <class 'sympy.core.symbol.Symbol'> | ()
returning arg from base case:
z <class 'str'>
[0] returned arg: z
--- 1 all func args: (z, 2)
recursion level start...
func: <class 'sympy.core.numbers.Integer'> | ()
ret

tensor(10.6930)

In [30]:
fn = lambda x, y: partial(ConcretizedCallable, call_name="add")(abstract_args=[x, y])

In [31]:
fn(*["x", "y"])

add['x', 'y'])

In [32]:
reduce(fn, ["x", "y", "z"])

add[add['x', 'y']), 'z'])

In [67]:
get_funcs(cc)

[pow] args: ['cos', 'log']
[cos] args: ['z']
final arg: z
---------------
[log] args: ['mul']
[mul] args: [1.5, 'pow']
independent arg: 1.5
---------------
[pow] args: ['add', -1.0]
[add] args: [1.0, 'mul']
independent arg: 1.0
---------------
[mul] args: [2.0, 'sin']
independent arg: 2.0
---------------
[sin] args: ['x']
final arg: x
---------------
independent arg: -1.0
---------------


tensor(1.0128)

In [22]:
op = pyq.Scale(pyq.X(0), sin_t)

embedding = pyq.Embedding(
    tparam_name="t",
    var_to_call={sin_t: sin_fn, t_sq: t_sq_fn},
)

In [23]:
vals = {"t": torch.tensor(0.2)}
generator.tensor(values=vals, embedding=embedding, full_support=[0,1]).reshape(4,4)

tensor([[0.0000+0.0000j, 0.0000-4.0000j, 7.9468+0.0000j, 0.0000+0.0000j],
        [0.0000+4.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 7.9468+0.0000j],
        [7.9468+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 0.0000-4.0000j],
        [0.0000+0.0000j, 7.9468+0.0000j, 0.0000+4.0000j, 0.0000+0.0000j]],
       dtype=torch.complex128)

In [14]:
embedded_params = embedding(vals)

In [15]:
op.tensor(values=embedded_params, embedding=embedding)

tensor([[[0.0000+0.j],
         [0.5985+0.j]],

        [[0.5985+0.j],
         [0.0000+0.j]]], dtype=torch.complex128)

In [15]:
from pyqtorch.primitives import Parametric

In [16]:
def is_parametrized(operation: pyq.Sequence) -> bool:
    params = []
    for m in operation.modules():
        if isinstance(m, (pyq.Scale, Parametric)):
            params.append(m.param_name)

    res = False
    if any([isinstance(p, str) for p in params]):
        res = True
    return res

In [17]:
is_parametrized(pyq.Sequence([pyq.RX(0, "x"), pyq.RY(1, "y")]))

True

In [17]:
a = (1, 2)

In [18]:
c, d = a

In [20]:
c

1

In [13]:
values = {"t": torch.tensor(0.3)}

embedded_params = embedding(values)


reembedded_params = embedding.reembed_tparam(embedded_params, torch.tensor(0.2))

generator.tensor(values=reembedded_params, embedding=embedding).reshape(4, 4)

tensor([[0.0000+0.0000j, 0.0000-4.0000j, 7.9468+0.0000j, 0.0000+0.0000j],
        [0.0000+4.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 7.9468+0.0000j],
        [7.9468+0.0000j, 0.0000+0.0000j, 0.0000+0.0000j, 0.0000-4.0000j],
        [0.0000+0.0000j, 7.9468+0.0000j, 0.0000+4.0000j, 0.0000+0.0000j]],
       dtype=torch.complex128)

In [20]:
duration = 0.5
n_steps = 1000

# simulate with torch-based solver
psi0_torch = zero_state(2).reshape(-1, 1)
t_points = torch.linspace(0, duration, n_steps)
state_torch = pyq.sesolve(hamiltonian_t, psi0_torch, t_points, SolverType.DP5_SE).states[-1]
print(state_torch)

tensor([[-0.0951+0.0000j],
        [-0.1566+0.0000j],
        [ 0.0000-0.5102j],
        [ 0.0000-0.8403j]], dtype=torch.complex128)


In [21]:
import torch
from pyqtorch import uniform_state, HamiltonianEvolution
from pyqtorch.matrices import DEFAULT_MATRIX_DTYPE

n_qubits = 2
qubit_targets = list(range(n_qubits))

# Random hermitian hamiltonian
matrix = torch.rand(2**n_qubits, 2**n_qubits, dtype=DEFAULT_MATRIX_DTYPE)
hermitian_matrix = matrix + matrix.T.conj()

time = torch.tensor([1.0])
time_symbol = "t"

hamiltonian_evolution = HamiltonianEvolution(generator=generator, time=time_symbol, qubit_support=qubit_targets, 
                                             is_time_dependent=True, duration=duration, steps=n_steps, solver=SolverType.DP5_SE)

# Starting from a uniform state
psi_start = zero_state(n_qubits)

# Returns the evolved state
psi_end = hamiltonian_evolution(state=psi_start, values={time_symbol: time}, embedding=embedding)

print(psi_end)

Taking support from generator and ignoring qubit_support input.


KeyError: 'sin_x'

tensor(0.1987)

In [1]:
import json

In [2]:
with open("/home/vytautas/Downloads/task_all_annotations_2024_08_26_07_25_47_coco 1 (1).0/annotations/instances_all.json", "r") as f:
    a = json.load(f)

In [6]:
a["categories"]

[{'id': 1, 'name': 'lav', 'supercategory': ''},
 {'id': 2, 'name': 'civ-car', 'supercategory': ''},
 {'id': 3, 'name': 'infantry', 'supercategory': ''},
 {'id': 4, 'name': 'mlrs', 'supercategory': ''},
 {'id': 5, 'name': 'Tank', 'supercategory': ''},
 {'id': 6, 'name': 'Truck', 'supercategory': ''},
 {'id': 7, 'name': 'apc', 'supercategory': ''},
 {'id': 8, 'name': 'army-truck', 'supercategory': ''},
 {'id': 9, 'name': 'artillery-gun', 'supercategory': ''},
 {'id': 10, 'name': 'car', 'supercategory': ''},
 {'id': 11, 'name': 'humvee', 'supercategory': ''},
 {'id': 12, 'name': 'military-vehicle', 'supercategory': ''},
 {'id': 13, 'name': 'person', 'supercategory': ''},
 {'id': 14, 'name': 'pickup', 'supercategory': ''},
 {'id': 15, 'name': 'rocket-artillery', 'supercategory': ''},
 {'id': 16, 'name': 'soldier', 'supercategory': ''},
 {'id': 17, 'name': 'tank', 'supercategory': ''},
 {'id': 18, 'name': 'truck', 'supercategory': ''},
 {'id': 19, 'name': 'van', 'supercategory': ''},
 {'id'

In [15]:
for annot in a["annotations"]:
    if annot["category_id"] == 10:
        annot["category_id"] = 2

In [16]:
with open("/home/vytautas/Downloads/task_all_annotations_2024_08_26_07_25_47_coco 1 (1).0/annotations/instances_all.json", "w") as f:
    json.dump(a, f)